###### this is as far as I could learn and fight the errors in quite some days , I don't have any idea how to fix the current error , when I'm changing the shape of bbox , it gives error that batch shape is different and if I try to match , it says batch shape is some different

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

C:\Users\devan\anaconda3\envs\py310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
(train_data, val_data), info = tfds.load(
    'coco/2017', 
    split=['train', 'validation'], 
    with_info=True, 
    data_dir='D:\Python_projects\Jupyter_lab\AIC\coco_dataset'  #directory to store the data
)

In [3]:
info

tfds.core.DatasetInfo(
    name='coco',
    full_name='coco/2017/1.1.0',
    description="""
    COCO is a large-scale object detection, segmentation, and
    captioning dataset.
    
    Note:
     * Some images from the train and validation sets don't have annotations.
     * Coco 2014 and 2017 uses the same images, but different train/val/test splits
     * The test split don't have any annotations (only images).
     * Coco defines 91 classes but the data only uses 80 classes.
     * Panotptic annotations defines defines 200 classes but only uses 133.
    """,
    config_description="""
    
    This version contains images, bounding boxes and labels for the 2017 version.
    
    """,
    homepage='http://cocodataset.org/#home',
    data_dir='D:\\Python_projects\\Jupyter_lab\\AIC\\coco_dataset\\coco\\2017\\1.1.0',
    file_format=tfrecord,
    download_size=25.20 GiB,
    dataset_size=24.98 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8)

In [4]:
vars(train_data)

{'_input_dataset': <ParallelMapDataset element_spec={'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'image/filename': TensorSpec(shape=(), dtype=tf.string, name=None), 'image/id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'objects': {'area': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'bbox': TensorSpec(shape=(None, 4), dtype=tf.float32, name=None), 'id': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'is_crowd': TensorSpec(shape=(None,), dtype=tf.bool, name=None), 'label': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}}>,
 '_buffer_size': <tf.Tensor: shape=(), dtype=int64, numpy=-1>,
 '_name': None,
 '_variant_tensor_attr': <tf.Tensor: shape=(), dtype=variant, value=<PrefetchDatasetOp::Dataset>>,
 '_graph_attr': <tensorflow.python.framework.ops.Graph at 0x22484512410>,
 '_options_attr': <tensorflow.python.data.ops.options.Options at 0x2248447f7c0>}

In [19]:
def preprocess_data(example):
    image = tf.image.convert_image_dtype(example['image'], dtype=tf.float32)
    image = tf.image.resize(image, (224, 224))  # Resize images for consistency
    image = tf.reshape(image, [224, 224, 3])
    image = tf.image.per_image_standardization(image)
    # Resize 'bboxes' tensor
    example['objects']['bbox'] = tf.reshape(example['objects']['bbox'], (3,4))
    
    # Extract bounding boxes and class labels
    bboxes = example['objects']['bbox']
    labels = example['objects']['label']
    
    return image, {'bboxes': bboxes, 'labels': labels}

In [27]:
# Apply the `preprocess_data` function to the datasets
train_data = train_data.map(lambda example: preprocess_data(example)).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
val_data = val_data.map(lambda example: preprocess_data(example)).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.<lambda>() takes 1 positional argument but 2 were given


In [21]:
# Define the CNN architecture
# Here, we use a pre-trained ResNet50 model and adapt it for object detection
base_model = keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

In [22]:
# Freeze the base model layers
base_model.trainable = False
num_classes = 80

In [23]:

# Add custom layers for object detection
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
bbox_outputs = layers.Dense(4, activation='linear', name='bboxes')(x)  # 4 outputs for bounding box coordinates
label_outputs = layers.Dense(num_classes, activation='softmax', name='labels')(x)  # Assuming num_classes is defined somewhere
outputs = {"bboxes": bbox_outputs, "labels": label_outputs}

In [24]:

model = keras.Model(inputs, outputs)

In [25]:
model.compile( optimizer=keras.optimizers.Adam(),
            loss={"bboxes": keras.losses.MeanSquaredError(), "labels": keras.losses.CategoricalCrossentropy()}, metrics={"bboxes": keras.metrics.MeanAbsoluteError(), "labels": keras.metrics.CategoricalAccuracy()}, )

In [26]:
# Train the model
epochs = 10
history = model.fit(train_data, epochs=epochs)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'IteratorGetNext' defined at (most recent call last):
    File "C:\Users\devan\anaconda3\envs\py310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\devan\anaconda3\envs\py310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\devan\anaconda3\envs\py310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\devan\anaconda3\envs\py310\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\devan\anaconda3\envs\py310\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\devan\anaconda3\envs\py310\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\devan\anaconda3\envs\py310\lib\asyn